In [16]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hvplot.pandas
import os
from pathlib import Path
from dotenv import load_dotenv
from pycoingecko import CoinGeckoAPI
%matplotlib inline

cg = CoinGeckoAPI()

In [17]:
# Initial imports
import os
import requests
import pandas as pd
import time
import datetime
#import alpaca_trade_api as tradeapi
#from dotenv import load_dotenv
from pycoingecko import CoinGeckoAPI
%matplotlib inline

cg = CoinGeckoAPI()

# Load .env environment variables
#load_dotenv()

# Set Alpaca API key and secret
#alpaca_api_key = os.getenv("ALPACA_API_KEY")
#alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
#alpaca = tradeapi.REST(
#    alpaca_api_key,
#    alpaca_secret_key,
#    api_version="v2")

## Initialize some dictionary and lists
portfolio_dict={}  # portfolio dictionary init
portfolio_list=[]  # list of portfolio tickers init
social_dict={} # dictionary for social data init
equities_dict={} #dictionary for equities data init
equities_list=[]





def get_crypto_portfolio_data(data):
### Uses get_price from the sdk and returns data for the users portfolio
    data = cg.get_price(ids=data,vs_currencies='usd', include_market_cap='true', include_24hr_vol='true', include_24hr_change='true', include_last_updated_at='false')
    return data


def get_coin_by_id(data):
### returns information about the coin including social data
    data = cg.get_coin_by_id(id=data,localization='false',tickers='false',market_data='false')
    return data


def create_crypto_portfolio_df(ticker_list):
##returns a portfolio dataframe of from a list of tickers
    
    #iterate through the list that was passed 
    for ticker in ticker_list:
        
        #get the data from the api
        api_data=get_crypto_portfolio_data(ticker)
    
        #the coin name is the first key in the api return data. We need it.
        coin = list(api_data.keys())[0] 

        #add data to dictionary
        portfolio_dict[coin] = api_data[coin]

    #Turn the dict into a data frame.
    df = pd.DataFrame.from_dict(portfolio_dict, orient='index')

    #wrangle the data into a portfolio
    df.columns=['Price', 'Market Cap', '24h Volume', '24h Change']
    df['Market Cap'] = df['Market Cap'].astype('int64')
    #df['24h Volume '] = df['24h Volume '].astype('int32')
    df['24h Change'] = df['24h Change'].astype('float64').round(decimals=2)

    #return the dataframe
    return df


def get_social_df(tickers):
# accepts a list of tickers and returns social inco in a dataframe

    for ticker in tickers:
        data=get_coin_by_id(ticker)
        social_dict[data['id']] = data['community_data']

    #Turn the dict into a data frame.
    df = pd.DataFrame.from_dict(pd.np.empty((0, 7)),social_dict)
    #df = pd.DataFrame(pd.np.empty((0, 7)))
    df.columns=['FB Likes', 'TWTR Follows', 'reddit posts 48h', 'reddit comments 48h','reddit subscribes 48h','reddit active 48h','telegram users']
        
    #return the dataframe
    return df



def get_equities_df(data):

    # Format current date as ISO format
    today = pd.Timestamp("2020-07-14", tz="America/New_York").isoformat()

    # Set timeframe to one day ('1D') for the Alpaca API
    timeframe = "1D"

    # Get current closing prices for FB and TWTR
    df_portfolio = alpaca.get_barset(
        tickers,
        timeframe,
        start = today,
        end = today
    ).df

    # Display sample data
    df_portfolio




def get_crypt_history(ticker_list, start_date = None, end_date = None):
    
    unix_start_date = int(time.mktime(start_date.timetuple()))
    unix_end_date = int(time.mktime(end_date.timetuple()))
    
    history_dfs = []

    for ticker in ticker_list:
        
        #Get the 5 year history 
        history = cg.get_coin_market_chart_range_by_id(id = ticker, vs_currency ='usd', from_timestamp=unix_start_date, to_timestamp=unix_end_date)  
        
        #wrangle the data
        prices = history['prices']
        market_caps = history['market_caps']
        total_volumes = history['total_volumes']
        
        #create the dataframes
        prices_df = pd.DataFrame(prices).set_index(0)
        market_caps_df = pd.DataFrame(market_caps).set_index(0)
        total_volumes_df = pd.DataFrame(total_volumes).set_index(0)
        
        #concat the dataframes
        joined = pd.concat([prices_df,market_caps_df,total_volumes_df], axis = 1, join = 'inner').reset_index()
        joined.columns = ['Date', 'Price', 'Market Caps', 'Total Volumes']
        joined['Date'] = pd.to_datetime(joined['Date'], unit = 'ms')
        joined.insert(loc=0, column='Ticker', value=ticker)
        history_dfs.append(joined)
    history_df=pd.concat(history_dfs)


    #return the 5 yr hostry dataframe
    return history_df
    


def get_user_input():

    while True:
        tickers = input("Please enter a ticker symbol. For equities prefix symbol with % (like %APPL).Make an empty entry when finished: ")

        #Check if a blank was entered. If so, end the dictionary creation.
        if tickers == '':
            break

        if tickers[0] == '%':
            equities_list.append(tickers)
            print("We can get stock data this way someday, but not yet....")
            continue

        try:
            #call the function to get crypto data
            api_data=get_crypto_portfolio_data(tickers)

            #Check that we have data. If not, inform users and continue while
            if not api_data:
                print(f"\t\t\t- {str(tickers).upper()} is not a valid symbol, please try again...")
                continue
            else:
                print(f"\t\t\t+ {str(tickers).upper()} has been added to the portfolio.")

            #append to the list of portfolio tickers for use elsewhere
            portfolio_list.append(tickers)

        except:
            pass


In [18]:
## Initialize some dictionary and lists
portfolio_dict={}  # portfolio dictionary init
portfolio_list=[]  # list of portfolio tickers init
social_dict={} # dictionary for social data init
equities_dict={} #dictionary for equities data init
equities_list=[]


In [19]:
social_df = get_social_df(portfolio_list)
social_df.to_csv('data_social.csv')
social_df.plot.bar(figsize=(20,7))


AttributeError: 'dict' object has no attribute 'lower'

In [20]:
# Import the CSVs to Pandas DataFrames
file_path = Path("test_data.csv")
test_data = pd.read_csv(file_path, index_col="name")
test_data

,price,market_cap,volume
name,,,
BTC,15258.00,282722040430,8907998000
ETH,400.00,50530052456,4024715113
LINK,12.50,4870588545,667605856
DOT,4.35,4053253172,173888050
S&P 500,3509.43,27000000000000,2168000000


In [21]:
test_plot = test_data.hvplot.bar(
    y="price")
test_plot

:Bars   [name]   (price)

In [22]:
correlation = test_data.corr()
correlation

,price,market_cap,volume
price,1.000000,-0.018449,0.907353
market_cap,-0.018449,1.000000,-0.152406
volume,0.907353,-0.152406,1.000000


In [23]:
volume = test_data.hvplot.line(
    x="name", y="volume"
)
volume

:Curve   [name]   (volume)

In [24]:
def get_social_df(tickers):
# accepts a list of tickers and returns social inco in a dataframe

    for ticker in tickers:
        data=get_coin_by_id(ticker)
        social_dict[data['id']] = data['community_data']

    #Turn the dict into a data frame.
    df = pd.DataFrame.from_dict(social_dict, orient='index')
    df.columns=['FB Likes', 'TWTR Follows', 'reddit posts 48h', 'reddit comments 48h','reddit subscribes 48h','reddit active 48h','telegram users']
        
    #return the dataframe
    return df

In [25]:
# Define Panel Visualization Functions
def test_bar():
    test_bar = test_data.hvplot.bar(y="price")
    fig_test_bar = plt.figure()
    plt.close(fig_test_bar)
    return test_bar

def volume():
    volume = test_data.hvplot.line(
        x="name", y="volume")
    fig_correlation_map = plt.figure()
    plt.close(fig_correlation_map)
    return volume


In [28]:
row_1 = pn.Row(
    test_bar()),
row_2 = pn.Row(
    volume(), test_bar()
)
  
column = pn.Column(
    "##Crypto Selection",
    "Welcome to the Crypto Analyzer. Input your crypto ticker symbols below to navigate " 
    "through a detailed portfolio analysis with benchmarks to popular market indices. "
    "When the analysis is complete, you will have a good idea of how your selection "
    "has performed historically as compared to other cryptocurrencies and other "
    "investments.",
    test_bar(),
    volume(),
    "https://freepngimg.com/thumb/bitcoin/59654-offering-exchange-"
    "initial-bitcoin-cryptocurrency-currency-digital.png",
    background='#f0f0f0', width=700, height=1000,
)



#title = pn.pane.Markdown(
    #"#Crypto Analyzer",
    #background="#969696", 
    #width=400, 
    #height=50
#)

test_dashboard = pn.Tabs(
    ("Overview"),
    #dif
    ("Crypto Selector", column),
    #pn.Spacer(background="gray", width=500, height=200),
    ("Returns and Market Cap", test_bar),
    ("Correlation", volume),
    ("Stand Dev Beta", test_bar),
    ("Fourth Result", volume),
    ("Conclusion"),
    dynamic=True,
    tabs_location="above"
)

test_dashboard_final = pn.Column(test_dashboard)
#pn.Column(test_dashboard)


In [29]:
test_dashboard_final.servable()


Column
    [0] Tabs(dynamic=True)
        [0] Markdown(str)
        [1] Column(background='#f0f0f0', height=1000, sizing_mode='fixed', width=700)
            [0] Markdown(str)
            [1] Markdown(str)
            [2] HoloViews(Bars)
            [3] HoloViews(Curve)
            [4] PNG(str)
        [2] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive03747')
        [3] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Curve, name='interactive03831')
        [4] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive03915')
        [5] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Curve, name='interactive03999')
        [6] Markdown(str)